<a href="https://colab.research.google.com/github/adigulalkari/SIH_Gender_classification/blob/main/Gender_Identification_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [10]:
!pip install opencv-python

In [ ]:
!pip install datasets

In [3]:
from datasets import load_dataset

ds = load_dataset("tuandunghcmut/PETA")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/19000 [00:00<?, ? examples/s]

In [53]:
import random
from datasets import DatasetDict
shuffled_ds = ds['train'].shuffle(seed=42)

sampled_ds = shuffled_ds.select(range(1000))

In [54]:

def add_gender_column(example):
    if example['personalMale'] == 1:
        example['gender_male'] = 1
    elif example['personalFemale'] == 1:
        example['gender_male'] = 0
    else:
        example['gender_male'] = None
    return example

sampled_ds = sampled_ds.map(add_gender_column)
print(sampled_ds.features)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

{'image': Image(mode=None, decode=True, id=None), 'image_name': Value(dtype='string', id=None), 'image_index': Value(dtype='uint16', id=None), 'dataset_index': Value(dtype='uint16', id=None), 'dataset_name': Value(dtype='string', id=None), 'global_unified_person_id': Value(dtype='uint16', id=None), 'local_dataset_person_id': Value(dtype='uint16', id=None), 'accessoryHairBand': Value(dtype='uint16', id=None), 'accessoryHat': Value(dtype='uint16', id=None), 'accessoryHeadphone': Value(dtype='uint16', id=None), 'accessoryKerchief': Value(dtype='uint16', id=None), 'accessoryMuffler': Value(dtype='uint16', id=None), 'accessoryNothing': Value(dtype='uint16', id=None), 'accessorySunglasses': Value(dtype='uint16', id=None), 'carryingBabyBuggy': Value(dtype='uint16', id=None), 'carryingBackpack': Value(dtype='uint16', id=None), 'carryingFolder': Value(dtype='uint16', id=None), 'carryingLuggageCase': Value(dtype='uint16', id=None), 'carryingMessengerBag': Value(dtype='uint16', id=None), 'carryin

In [55]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(sampled_ds['image'], sampled_ds['gender_male'], test_size=0.2, random_state=42)

In [56]:
type(X_train[10])

PIL.BmpImagePlugin.BmpImageFile

In [57]:
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import preprocess_input
import numpy as np
from PIL import Image

IMAGE_SIZE = (224, 224)

def load_and_preprocess_image(img):
    if not isinstance(img, Image.Image):
        raise TypeError(f"Expected PIL Image, got {type(img)}")

    img = img.resize(IMAGE_SIZE, Image.Resampling.LANCZOS)
    img_array = np.array(img)

    if img_array.ndim == 2:  # Handle grayscale images
        img_array = np.expand_dims(img_array, axis=-1)
        img_array = np.repeat(img_array, 3, axis=-1)

    img_array = preprocess_input(img_array)

    return img_array

def preprocess_images(images):
    preprocessed_images = []
    for img in images:
        try:
            preprocessed_image = load_and_preprocess_image(img)
            preprocessed_images.append(preprocessed_image)
        except TypeError:
            print(f"Skipping invalid image of type {type(img)}")

    return np.array(preprocessed_images)

print("Preprocessing training images...")
X_train_preprocessed = preprocess_images(X_train)

print("Preprocessing test images...")
X_test_preprocessed = preprocess_images(X_test)

print(f"Shape of preprocessed training images: {X_train_preprocessed.shape}")
print(f"Shape of preprocessed test images: {X_test_preprocessed.shape}")


Preprocessing training images...
Preprocessing test images...
Shape of preprocessed training images: (800, 224, 224, 3)
Shape of preprocessed test images: (200, 224, 224, 3)


In [58]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)


In [59]:
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 230, 230, 3)    │              0 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 112, 112, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 112, 112, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 112, 112, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 114, 114, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 56, 56, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 56, 56, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 56, 56, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 56, 56, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 56, 56, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)             

 Total params: 25,686,913 (97.99 MB)

 Trainable params: 2,099,201 (8.01 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [60]:
print(type(X_train_preprocessed))
print(X_train_preprocessed.shape)


<class 'numpy.ndarray'>
(800, 224, 224, 3)


In [61]:
y_train = np.array(y_train)
y_test = np.array(y_test)

print(y_train.shape)
print(y_test.shape)


(800,)
(200,)


In [62]:
history = model.fit(X_train_preprocessed, y_train,
                    validation_data=(X_test_preprocessed, y_test),
                    epochs=50,
                    batch_size=32)



Epoch 1/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 17s 358ms/step - accuracy: 0.5466 - loss: 0.7951 - val_accuracy: 0.6600 - val_loss: 0.6224
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 12s 109ms/step - accuracy: 0.7490 - loss: 0.5176 - val_accuracy: 0.6500 - val_loss: 0.6215
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 108ms/step - accuracy: 0.7930 - loss: 0.4427 - val_accuracy: 0.6850 - val_loss: 0.5901
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 116ms/step - accuracy: 0.8935 - loss: 0.3287 - val_accuracy: 0.6700 - val_loss: 0.5918
Epoch 5/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 110ms/step - accuracy: 0.9259 - loss: 0.3060 - val_accuracy: 0.6550 - val_loss: 0.5898
Epoch 6/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 114ms/step - accuracy: 0.9012 - loss: 0.2870 - val_accuracy: 0.6900 - val_loss: 0.6258
Epoch 7/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 119ms/step - accuracy: 0.9513 - loss: 0.2401 - val_accuracy: 0.6650 - val_loss: 0.6083
Epoch 8/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 114ms/step - accuracy: 0.9716 - loss: 0.2094 - val_accuracy: 

In [63]:
test_loss, test_acc = model.evaluate(X_test_preprocessed, y_test)
print(f"Test accuracy: {test_acc}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 77ms/step - accuracy: 0.6898 - loss: 0.8471
Test accuracy: 0.7049999833106995


In [65]:
model.save_weights('res_net_model.weights.h5')

In [66]:
!git remote add origin https://github.com/adigulalkari/SIH_Gender_classification.git

fatal: not a git repository (or any of the parent directories): .git


In [73]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout


base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


base_model.trainable = False


x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

In [74]:
model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [75]:
history = model.fit(X_train_preprocessed, y_train,
                    validation_data=(X_test_preprocessed, y_test),
                    epochs=50,
                    batch_size=32)


Epoch 1/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 29s 473ms/step - accuracy: 0.5704 - loss: 16.3125 - val_accuracy: 0.7250 - val_loss: 5.2602
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 18s 187ms/step - accuracy: 0.8109 - loss: 2.1691 - val_accuracy: 0.7800 - val_loss: 2.0958
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 188ms/step - accuracy: 0.9003 - loss: 0.4157 - val_accuracy: 0.7750 - val_loss: 1.1940
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 174ms/step - accuracy: 0.9397 - loss: 0.1957 - val_accuracy: 0.7800 - val_loss: 1.0088
Epoch 5/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 177ms/step - accuracy: 0.9778 - loss: 0.0670 - val_accuracy: 0.7800 - val_loss: 1.0428
Epoch 6/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 195ms/step - accuracy: 0.9917 - loss: 0.0272 - val_accuracy: 0.7750 - val_loss: 1.2295
Epoch 7/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 195ms/step - accuracy: 0.9930 - loss: 0.0337 - val_accuracy: 0.7900 - val_loss: 1.1402
Epoch 8/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 195ms/step - accuracy: 0.9918 - loss: 0.0218 - val_accuracy:

In [76]:
test_loss, test_acc = model.evaluate(X_test_preprocessed, y_test)
print(f"Test accuracy: {test_acc}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 122ms/step - accuracy: 0.7086 - loss: 3.1590
Test accuracy: 0.7400000095367432


In [77]:
model.save_weights('vgg_model.weights.h5')

In [78]:
import struct
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import ZeroPadding2D
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.layers import add, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import load_img
import matplotlib.pyplot as plt
from matplotlib import pyplot
from matplotlib.patches import Rectangle
from numpy import expand_dims

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt


In [96]:
import torch
import torchvision
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F
from PIL import Image
import cv2
from google.colab.patches import cv2_imshow

In [93]:
import torch
from yolov5.models.yolo import Model

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True).to(device)
model.eval()

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 v7.0-365-g12b577c8 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

100%|██████████| 14.1M/14.1M [00:00<00:00, 181MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  

In [ ]:
!pip install inference-sdk

In [102]:
from inference_sdk import InferenceHTTPClient

CLIENT = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key="lrHjGWuNWWiEAqJOyZ95"
)

result = CLIENT.infer('https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRjIODn9ZEde9svkuy94dxsCm-RZ8TUzjCrfA&s', model_id="people-detection-o4rdr/7")

In [103]:
result

{'inference_id': '20fe8b6b-ed83-4da9-a9db-5154f85fc7ba',
 'time': 0.04080577199988511,
 'image': {'width': 272, 'height': 185},
 'predictions': [{'x': 166.5,
   'y': 110.5,
   'width': 45.0,
   'height': 135.0,
   'confidence': 0.8809011578559875,
   'class': 'person',
   'class_id': 0,
   'detection_id': 'f3550680-b3b6-4e87-8194-65aa1857b933'},
  {'x': 127.0,
   'y': 107.0,
   'width': 38.0,
   'height': 134.0,
   'confidence': 0.8636706471443176,
   'class': 'person',
   'class_id': 0,
   'detection_id': '07eb0146-5bf8-4fbf-b2d1-3465c2c3997b'}]}

In [108]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
import numpy as np
from PIL import Image
import requests
from io import BytesIO

def load_vgg_model(weights_path):
    base_model = VGG16(weights=None, include_top=False, input_shape=(224, 224, 3))

    base_model.load_weights(weights_path)

    return base_model

def load_image_from_url(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content)).convert('RGB')
    return img

def preprocess_image(img):
    img = img.resize((224, 224))
    img_array = np.array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

def get_cropped_images(image, detections):
    cropped_images = []
    for det in detections:
        x = int(det['x'])
        y = int(det['y'])
        width = int(det['width'])
        height = int(det['height'])
        box = (x, y, x + width, y + height)
        cropped_image = image.crop(box)
        cropped_images.append(cropped_image)
    return cropped_images

def infer_gender(model, cropped_images):
    for img in cropped_images:
        img_array = preprocess_image(img)
        predictions = model.predict(img_array)
        decoded_predictions = decode_predictions(predictions)[0]
        print(decoded_predictions)

def main(image_url, weights_path):
    image = load_image_from_url(image_url)
    detection_results = [
        {'x': 166.5, 'y': 110.5, 'width': 45.0, 'height': 135.0},
        {'x': 127.0, 'y': 107.0, 'width': 38.0, 'height': 134.0}
    ]
    model = load_vgg_model(weights_path)

    cropped_images = get_cropped_images(image, detection_results)

    infer_gender(model, cropped_images)


image_url = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRjIODn9ZEde9svkuy94dxsCm-RZ8TUzjCrfA&s'
weights_path = '/content/vgg_model.weights.h5'
main(image_url, weights_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 313ms/step


ValueError: `decode_predictions` expects a batch of predictions (i.e. a 2D array of shape (samples, 1000)). Received array with shape: (1, 7, 7, 512)